In [ ]:
# Fetch hourly weather info through open-meteo API

import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry
import json
from datetime import datetime, timezone

def weatherapi(latitude, longitude,location_name):
    # Setup session with caching and retry logic
    cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    client = openmeteo_requests.Client(session=retry_session)

    # API endpoint and parameters
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": [
            "temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature","wind_speed_10m",
            "precipitation_probability", "precipitation", "rain", "showers", "snowfall",
            "snow_depth", "weather_code", "pressure_msl", "surface_pressure", "cloud_cover",
            "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "visibility",
            "evapotranspiration", "et0_fao_evapotranspiration", "vapour_pressure_deficit"
        ]
    }

    # Fetch weather data
    responses = client.weather_api(url, params=params)
    response = responses[0]
    hourly = response.Hourly()

    # Generate datetime range for all hourly data points
    time_range = pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    )

    # Find current UTC hour rounded down
    now_utc = datetime.now(timezone.utc).replace(minute=0, second=0, microsecond=0)

    # Find the index of the current hour in the time_range
    try:
        current_index = time_range.get_loc(now_utc)
    except KeyError:
        # If current time not in forecast range, fallback to nearest time (first available)
        current_index = 0

    # Prepare data for the current hour only
    current_hour_data = {"city": location_name}

    variables = [
        "temperature_celcius", "humidity_%", "dew_temperature_celcius", "feels_like_temperature_celcius","wind_speed_kmph",
        "precipitation_%", "precipitation_occured_mm", "rain_mm", "showers_mm", "snowfall_mm",
        "snow_depth_mm", "weather_code", "mean_sea_level_pressure_hpa", "surface_pressure_hpa", "cloud_cover_%",
        "visibility_m",
        "evapotranspiration_mm", "et0_fao_evapotranspiration_mm", "vapour_pressure_deficit_kpa"
    ]

    # Extract the value for the current hour from each variable
    for i, var in enumerate(variables):
        values = hourly.Variables(i).ValuesAsNumpy()
        current_hour_data[var] = float(values[current_index]) if values.size > current_index else None

    return current_hour_data

def fetch_weather_batch():
    delhi=weatherapi(28.7041,77.1025,'Delhi')

    mumbai=weatherapi(18.9582,72.8321,'Mumbai')

    bengaluru=weatherapi(12.9629,77.5775,'Bengaluru')

    hyderabad=weatherapi(17.4065, 78.4772,'Hyderabad')

    chennai=weatherapi(13.0843,80.2705,'Chennai')

    kolkata=weatherapi(22.5744,88.3629,'Kolkata')

    ahmedabad=weatherapi(23.0225,72.5714,'Ahmedabad')

    pune=weatherapi(18.5246,73.8786,'Pune')

    jaipur=weatherapi(26.9124,75.7873,'Jaipur')

    lucknow=weatherapi(26.8467,80.9462,'Lucknow')
    
    all_cities=[delhi,mumbai,bengaluru,hyderabad,chennai,kolkata,ahmedabad,pune,jaipur,lucknow]

    # debugging
    print(all_cities)


    return all_cities

fetch_weather_batch()





"""
    Fetches current hourly weather data for a specified location using the Open-Meteo API.
    
    Args:
        latitude (float): Latitude of the location.
        longitude (float): Longitude of the location.
        location_name (str): Human-readable name of the location (for reference).
        
    Returns:
        dict: Weather data for the current hour, including temperature, humidity, precipitation,
              cloud cover, and other atmospheric variables.
              
    Implementation details:
        - Uses a cached session with automatic retries to improve API request reliability.
        - Fetches hourly forecast data from Open-Meteo API for a wide range of weather variables.
        - Converts API response timestamps to pandas datetime for easier time handling.
        - Extracts the data corresponding to the current UTC hour.
        - Returns a dictionary with all requested weather parameters for the current hour.

        
Data Dictionary Explanation:

- location: Name of the city (string).
- date: Date and time of the weather reading in 'YYYY-MM-DD HH:MM:SS' format (string).
- temperature_2m: Air temperature measured 2 meters above the ground, in degrees Celsius (float).
- relative_humidity_2m: Relative humidity at 2 meters height, in percentage (%) (float).
- dew_point_2m: Dew point temperature at 2 meters height, in degrees Celsius (float).
- apparent_temperature: Feels-like temperature considering humidity and wind, in degrees Celsius (float).
- wind_speed_10m: wind speed in kmph above 10 meter of ground level(float).
- precipitation_probability: Probability of precipitation occurring, in percentage (%) (float).
- precipitation: Amount of precipitation expected or recorded, in millimeters (mm) (float).
- rain: Amount of rain specifically, in millimeters (mm) (float).
- showers: Amount of showers, in millimeters (mm) (float).
- snowfall: Amount of snowfall, in millimeters (mm) (float).
- snow_depth: Depth of snow on the ground, in millimeters (mm) (float).
- weather_code: Numerical code representing weather conditions (e.g., clear, rain, snow) (float).
- pressure_msl: Atmospheric pressure at mean sea level, in hectopascals (hPa) (float).
- surface_pressure: Atmospheric pressure at the surface, in hectopascals (hPa) (float).
- cloud_cover: Total cloud cover percentage (%) (float).
- cloud_cover_low: Low-level cloud cover percentage (%) (float).
- cloud_cover_mid: Mid-level cloud cover percentage (%) (float).
- cloud_cover_high: High-level cloud cover percentage (%) (float).
- visibility: Visibility distance in meters (float).
- evapotranspiration: Amount of evapotranspiration, representing water transfer from land to atmosphere, in millimeters (mm) (float).
- et0_fao_evapotranspiration: Reference evapotranspiration (FAO standard), in millimeters (mm) (float).
- vapour_pressure_deficit: Difference between saturation and actual vapor pressure, indicating dryness of air, in kilopascals (kPa) (float).

"""


[{'city': 'Delhi', 'temperature_celcius': 27.62849998474121, 'humidity_%': 89.0, 'dew_temperature_celcius': 25.650005340576172, 'feels_like_temperature_celcius': 33.98313903808594, 'wind_speed_kmph': 5.58870267868042, 'precipitation_%': 70.0, 'precipitation_occured_mm': 2.5999999046325684, 'rain_mm': 0.10000000149011612, 'showers_mm': 2.5999999046325684, 'snowfall_mm': 0.0, 'snow_depth_mm': 0.0, 'weather_code': 96.0, 'mean_sea_level_pressure_hpa': 997.0999755859375, 'surface_pressure_hpa': 972.3311767578125, 'cloud_cover_%': 100.0, 'visibility_m': 77.0, 'evapotranspiration_mm': 60.0, 'et0_fao_evapotranspiration_mm': 100.0, 'vapour_pressure_deficit_kpa': 8920.0}, {'city': 'Mumbai', 'temperature_celcius': 28.01500129699707, 'humidity_%': 86.0, 'dew_temperature_celcius': 25.45258140563965, 'feels_like_temperature_celcius': 31.292449951171875, 'wind_speed_kmph': 25.772077560424805, 'precipitation_%': 100.0, 'precipitation_occured_mm': 1.0, 'rain_mm': 0.0, 'showers_mm': 1.0, 'snowfall_mm': 

"\n    Fetches current hourly weather data for a specified location using the Open-Meteo API.\n    \n    Args:\n        latitude (float): Latitude of the location.\n        longitude (float): Longitude of the location.\n        location_name (str): Human-readable name of the location (for reference).\n        \n    Returns:\n        dict: Weather data for the current hour, including temperature, humidity, precipitation,\n              cloud cover, and other atmospheric variables.\n              \n    Implementation details:\n        - Uses a cached session with automatic retries to improve API request reliability.\n        - Fetches hourly forecast data from Open-Meteo API for a wide range of weather variables.\n        - Converts API response timestamps to pandas datetime for easier time handling.\n        - Extracts the data corresponding to the current UTC hour.\n        - Returns a dictionary with all requested weather parameters for the current hour.\n\n        \nData Dictionary E

In [357]:
import pandas as pd
import random
from datetime import datetime
# from weather_monitor import weatherapi,fetch_weather_batch
df = pd.DataFrame(fetch_weather_batch())



[{'city': 'Delhi', 'temperature_celcius': 27.928499221801758, 'humidity_%': 89.0, 'dew_temperature_celcius': 25.945632934570312, 'feels_like_temperature_celcius': 34.49635696411133, 'wind_speed_kmph': 5.506940841674805, 'precipitation_%': 63.0, 'precipitation_occured_mm': 3.799999952316284, 'rain_mm': 0.0, 'showers_mm': 3.799999952316284, 'snowfall_mm': 0.0, 'snow_depth_mm': 0.0, 'weather_code': 96.0, 'mean_sea_level_pressure_hpa': 996.2000122070312, 'surface_pressure_hpa': 971.477783203125, 'cloud_cover_%': 100.0, 'visibility_m': 74.0, 'evapotranspiration_mm': 53.0, 'et0_fao_evapotranspiration_mm': 100.0, 'vapour_pressure_deficit_kpa': 5340.0}, {'city': 'Mumbai', 'temperature_celcius': 27.96500015258789, 'humidity_%': 86.0, 'dew_temperature_celcius': 25.403520584106445, 'feels_like_temperature_celcius': 31.733489990234375, 'wind_speed_kmph': 22.180206298828125, 'precipitation_%': 98.0, 'precipitation_occured_mm': 1.2999999523162842, 'rain_mm': 0.0, 'showers_mm': 1.2999999523162842, 's

In [319]:
# DATA CLEANING

#rounding off all float cols
df = df.round({
    'temperature_celcius': 2,
    'humidity_%': 0,
    'dew_temperature_celcius': 2,
    'feels_like_temperature_celcius': 2,
    'wind_speed_kmph':2,
    'precipitation_%': 0,
    'precipitation_occured_mm': 2,
    'rain_mm': 2,
    'showers_mm': 2,
    'snowfall_mm': 2,
    'snow_depth_mm': 2,
    'mean_sea_level_pressure_hpa': 2,
    'surface_pressure_hpa': 1,
    'cloud_cover_%': 0,
    'visibility_m': 2,
    'evapotranspiration_mm': 3,
    'et0_fao_evapotranspiration_mm': 3,
    'vapour_pressure_deficit_kpa': 4
})


#missing values check
critical_columns = [
    "temperature_celcius", "humidity_%", "dew_temperature_celcius", "feels_like_temperature_celcius",
    "wind_speed_kmph", "precipitation_%", "precipitation_occured_mm", "rain_mm", "showers_mm", "snowfall_mm",
    "snow_depth_mm", "weather_code", "mean_sea_level_pressure_hpa", "surface_pressure_hpa", "cloud_cover_%",
    "visibility_m", "evapotranspiration_mm", "et0_fao_evapotranspiration_mm", "vapour_pressure_deficit_kpa"
]
df['is_missing_data'] = df[critical_columns].isnull().any(axis=1)


#outliers check
thresholds = {
    "temperature_celcius": (-90, 60),
    "humidity_%": (0, 100),
    "dew_temperature_celcius": (-100, 60),
    "feels_like_temperature_celcius": (-100, 70),
    "wind_speed_kmph": (0, 300),
    "precipitation_%": (0, 100),
    "precipitation_occured_mm": (0, 500),
    "rain_mm": (0, 500),
    "showers_mm": (0, 500),
    "snowfall_mm": (0, 500),
    "snow_depth_mm": (0, 1000),
    "weather_code": (0, 101),
    "cloud_cover_%": (0, 100),
    "visibility_m": (0, 100000)
    # "evapotranspiration_mm": (0, 50),
    # "et0_fao_evapotranspiration_mm": (0, 50),
    # "vapour_pressure_deficit_kpa": (0, 10),
    # "mean_sea_level_pressure_hpa": (870, 1085),
    # "surface_pressure_hpa": (870, 1085)
}
df['is_outlier'] = False
for col, (min_val, max_val) in thresholds.items():
    if col in df.columns:
        df['is_outlier'] |= (df[col] < min_val) | (df[col] > max_val)


#delete if multiple data coming from same city.
df.drop_duplicates(subset=['city'], inplace=True)

#audit trial cols
df['processing_status'] = 'cleaned'


#drop if imp cols have null values
df.dropna(subset=["city","temperature_celcius","weather_code"],inplace=True)

In [320]:
# DATA TRANSFORMATION 


# Add current datetime in the specified format
df['created_at'] = datetime.now()


# Add water strees index
df['water_stress_index'] = (
    df['evapotranspiration_mm'] / (df['precipitation_occured_mm'] + 0.01)  # Avoid div/0
).round(2)



# Add column of check extreme condition or not
extreme_codes = [] #list of dangerous/extreme weather codes to be given inside the list
if extreme_codes:
    df['extreme_weather_yn'] = df['weather_code'].isin(extreme_codes).map({True: 'Y', False: 'N'})
else:
    df['extreme_weather_yn'] = 'N/A'



# is_snowfall column
df['is_snowfall'] = df['snowfall_mm'] > 0



# is_rainfall column
df['is_rainfall'] = (df['rain_mm'] > 0) | (df['showers_mm'] > 0)



# is_foggy column
df['is_foggy'] = df['visibility_m'] < 1000



# effective_precipitation_mm based on sum of snowfall,rain and shower i mm
SNOW_TO_LIQUID_RATIO = 10
# Convert snowfall_mm to its water equivalent
df['snowfall_water_equivalent_mm'] = df['snowfall_mm'] / SNOW_TO_LIQUID_RATIO
df['effective_precipitation_mm'] = df['rain_mm'] + df['showers_mm'] + df['snowfall_water_equivalent_mm']


# creating unique_id
city_short_map = {
    'Delhi': 'DEL', 
    'Mumbai': 'MUM', 
    'Bengaluru': 'BLR', 
    'Hyderabad': 'HYD', 
    'Chennai': 'CHE',  
    'Kolkata': 'KOL',  
    'Ahmedabad': 'AMD',  
    'Pune': 'PUN',  
    'Jaipur': 'JAI', 
    'Lucknow': 'LKO'  
}
def generate_unique_id(row):
    city_short = city_short_map.get(row['city'], row['city'][:3].upper())
    weather_code_str = f"{int(row['weather_code']):02d}"  # pad weather code to 2 digits
    rand_num = f"{random.randint(0, 99):02d}"            # 2 digit random number
    dt = row['created_at']
    dt_str = dt.strftime('%d%m%y%H%M%S')                # DDMMYYHHMMSS 
    ms_str = f"{int(dt.microsecond / 10000):02d}"        # first 2 digits of milliseconds  
    unique_id = f"{city_short}-{weather_code_str}-{rand_num}-{dt_str}{ms_str}"
    return unique_id
df['unique_id'] = df.apply(generate_unique_id, axis=1)
#unique id = 3 leters city shortform-weather_code-random 2 digit-date time in format DDMMYYHHMMSSMS




#weather type column based on weather code (WMO standard)
# Step 1: Create the mapping dictionary
weather_code_map = {
    0: "Clear sky",
    1: "Mainly clear",
    2: "Partly cloudy",
    3: "Overcast",
    4: "Fog",
    5: "Drizzle",
    6: "Rain",
    7: "Showers",
    8: "Snow",
    9: "Rain and snow",
    10: "Sleet",
    11: "Hail",
    12: "Thunderstorm",
    13: "Duststorm",
    14: "Sandstorm",
    15: "Smoke",
    16: "Volcanic ash",
    17: "Windstorm",
    18: "Tornado",
    19: "Freezing rain",
    20: "Mist",
    21: "Light rain",
    22: "Moderate rain",
    23: "Heavy rain",
    24: "Light snow",
    25: "Moderate snow",
    26: "Heavy snow",
    27: "Light sleet",
    28: "Moderate sleet",
    29: "Heavy sleet",
    30: "Light hail",
    31: "Moderate hail",
    32: "Heavy hail",
    33: "Light thunderstorm",
    34: "Moderate thunderstorm",
    35: "Heavy thunderstorm",
    36: "Light duststorm",
    37: "Moderate duststorm",
    38: "Heavy duststorm",
    39: "Light sandstorm",
    40: "Moderate sandstorm",
    41: "Heavy sandstorm",
    42: "Light smoke",
    43: "Moderate smoke",
    44: "Heavy smoke",
    45: "Light volcanic ash",
    46: "Moderate volcanic ash",
    47: "Heavy volcanic ash",
    48: "Light windstorm",
    49: "Strong windstorm",
    50: "Severe windstorm",
    51: "Light tornado",
    52: "Moderate tornado",
    53: "Heavy tornado",
    54: "Light freezing rain",
    55: "Moderate freezing rain",
    56: "Heavy freezing rain",
    57: "Light mist",
    58: "Moderate mist",
    59: "Heavy mist",
    60: "Light rain and snow",
    61: "Moderate rain and snow",
    62: "Heavy rain and snow",
    63: "Light sleet and snow",
    64: "Moderate sleet and snow",
    65: "Heavy sleet and snow",
    66: "Light hail and snow",
    67: "Moderate hail and snow",
    68: "Heavy hail and snow",
    69: "Light thunderstorm with rain",
    70: "Moderate thunderstorm with rain",
    71: "Heavy thunderstorm with rain",
    72: "Light thunderstorm with snow",
    73: "Moderate thunderstorm with snow",
    74: "Heavy thunderstorm with snow",
    75: "Light thunderstorm with sleet",
    76: "Moderate thunderstorm with sleet",
    77: "Heavy thunderstorm with sleet",
    78: "Light thunderstorm with hail",
    79: "Moderate thunderstorm with hail",
    80: "Heavy thunderstorm with hail",
    81: "Light thunderstorm with dust",
    82: "Moderate thunderstorm with dust",
    83: "Heavy thunderstorm with dust",
    84: "Light thunderstorm with sand",
    85: "Moderate thunderstorm with sand",
    86: "Heavy thunderstorm with sand",
    87: "Light thunderstorm with smoke",
    88: "Moderate thunderstorm with smoke",
    89: "Heavy thunderstorm with smoke",
    90: "Light thunderstorm with volcanic ash",
    91: "Moderate thunderstorm with volcanic ash",
    92: "Heavy thunderstorm with volcanic ash",
    93: "Light windstorm with rain",
    94: "Strong windstorm with rain",
    95: "Severe windstorm with rain",
    96: "Light windstorm with snow",
    97: "Strong windstorm with snow",
    98: "Severe windstorm with snow",
    99: "Light windstorm with sleet",
    100: "Severe windstorm with sleet"
}
df['weather_type'] = df['weather_code'].map(weather_code_map)



# Soft delete if imp columns have null values
columns_imp = [
    'city', 'temperature_celcius', 'humidity_%', 'wind_speed_kmph',
    'weather_code', 'precipitation_occured_mm', 'created_at', 'extreme_weather_yn'
]
df['is_deleted'] = df[columns_imp].isnull().any(axis=1)



# Add audit trail columns
df['processing_status'] = 'transformed'
df['batch_id'] = datetime.now().strftime('%Y%m%d%H%M%S') + f"{datetime.now().microsecond // 1000:03d}"



#ordering columns
sorted_columns = [
    # Location Info
    'city',   
    # Core Weather Measurements
    'temperature_celcius',
    'feels_like_temperature_celcius',
    'dew_temperature_celcius',
    'humidity_%',
    'vapour_pressure_deficit_kpa',
    'wind_speed_kmph',    
    # Precipitation & Snowfall
    'precipitation_%',
    'precipitation_occured_mm',
    'rain_mm',
    'showers_mm',
    'is_rainfall',
    'snowfall_mm',
    'snow_depth_mm',
    'is_snowfall',
    'effective_precipitation_mm',
    # Fog & Visibility
    'is_foggy',
    'visibility_m',
    'cloud_cover_%',
    # Pressure & Wind
    'mean_sea_level_pressure_hpa',
    'surface_pressure_hpa',
    # Evaporation
    'evapotranspiration_mm',
    'et0_fao_evapotranspiration_mm',
    # Weather Code & Classification
    'weather_code',
    'weather_type',   
    # Environmental Indicators
    'water_stress_index',
    'extreme_weather_yn',
    # Timestamps
    'unique_id',
    'is_deleted',
    'created_at',
    # data info
    'is_missing_data',
    'is_outlier',
    'processing_status',
    'batch_id'

]
df = df[sorted_columns]

# for debugging
# print(df)

In [321]:
# Views dataset


import pandas as pd

# Show more columns
pd.set_option('display.max_columns', None)

# Optionally, show more rows (if you also need that)
pd.set_option('display.max_rows', None)

# Adjust column width (optional)
pd.set_option('display.max_colwidth', None)

# Prevent horizontal truncation
pd.set_option('display.width', None)  # Auto-detect terminal width

df
# df.isnull().sum()
# df.dtypes
# print("Total columns:", len(df.columns))


,city,temperature_celcius,feels_like_temperature_celcius,dew_temperature_celcius,humidity_%,vapour_pressure_deficit_kpa,wind_speed_kmph,precipitation_%,precipitation_occured_mm,rain_mm,showers_mm,is_rainfall,snowfall_mm,snow_depth_mm,is_snowfall,effective_precipitation_mm,is_foggy,visibility_m,cloud_cover_%,mean_sea_level_pressure_hpa,surface_pressure_hpa,evapotranspiration_mm,et0_fao_evapotranspiration_mm,weather_code,weather_type,water_stress_index,extreme_weather_yn,unique_id,is_deleted,created_at,is_missing_data,is_outlier,processing_status,batch_id
0,Delhi,27.63,33.98,25.65,89.0,8920.0,5.59,70.0,2.6,0.1,2.6,True,0.0,0.0,False,2.7,True,77.0,100.0,997.1,972.3,60.0,100.0,96.0,Light windstorm with snow,22.99,N/A,DEL-96-24-29062502284729,False,2025-06-29 02:28:47.294467,False,False,transformed,20250629022847300
1,Mumbai,28.02,31.29,25.45,86.0,23840.0,25.77,100.0,1.0,0.0,1.0,True,0.0,0.0,False,1.0,True,77.0,99.0,1003.8,1002.7,53.0,93.0,80.0,Heavy thunderstorm with hail,52.48,N/A,MUM-80-58-29062502284729,False,2025-06-29 02:28:47.294467,False,False,transformed,20250629022847300
2,Bengaluru,21.31,23.29,19.78,91.0,24140.0,12.18,0.0,0.0,0.0,0.0,False,0.0,0.0,False,0.0,True,70.0,100.0,1010.3,911.1,0.0,100.0,3.0,Overcast,0.00,N/A,BLR-03-13-29062502284729,False,2025-06-29 02:28:47.294467,False,False,transformed,20250629022847300
3,Hyderabad,25.01,27.77,21.52,81.0,24140.0,13.05,0.0,0.0,0.0,0.0,False,0.0,0.0,False,0.0,True,35.0,100.0,1005.2,947.6,0.0,100.0,3.0,Overcast,0.00,N/A,HYD-03-26-29062502284729,False,2025-06-29 02:28:47.294467,False,False,transformed,20250629022847300
4,Chennai,27.83,33.05,23.85,79.0,24140.0,5.35,15.0,0.0,0.0,0.0,False,0.0,0.0,False,0.0,True,0.0,100.0,1004.9,1003.6,81.0,100.0,3.0,Overcast,8100.00,N/A,CHE-03-30-29062502284729,False,2025-06-29 02:28:47.294467,False,False,transformed,20250629022847300
5,Kolkata,26.21,32.86,25.52,96.0,6900.0,2.97,78.0,0.5,0.0,0.5,True,0.0,0.0,False,0.5,True,84.0,97.0,996.7,995.6,67.0,45.0,80.0,Heavy thunderstorm with hail,131.37,N/A,KOL-80-96-29062502284729,False,2025-06-29 02:28:47.294467,False,False,transformed,20250629022847300
6,Ahmedabad,26.71,32.02,25.30,92.0,24140.0,11.09,28.0,0.4,0.0,0.4,True,0.0,0.0,False,0.4,True,87.0,100.0,999.2,993.2,92.0,100.0,80.0,Heavy thunderstorm with hail,224.39,N/A,AMD-80-79-29062502284729,False,2025-06-29 02:28:47.294467,False,False,transformed,20250629022847300
7,Pune,24.02,26.88,20.95,83.0,24140.0,10.25,15.0,0.0,0.0,0.0,False,0.0,0.0,False,0.0,True,81.0,100.0,1005.6,942.3,46.0,100.0,3.0,Overcast,4600.00,N/A,PUN-03-94-29062502284729,False,2025-06-29 02:28:47.294467,False,False,transformed,20250629022847300
8,Jaipur,26.79,32.31,24.82,89.0,24140.0,7.57,3.0,0.0,0.0,0.0,False,0.0,0.0,False,0.0,True,51.0,100.0,997.8,950.2,18.0,100.0,3.0,Overcast,1800.00,N/A,JAI-03-73-29062502284729,False,2025-06-29 02:28:47.294467,False,False,transformed,20250629022847300
9,Lucknow,28.02,34.83,25.65,87.0,21580.0,2.41,55.0,0.6,0.0,0.6,True,0.0,0.0,False,0.6,True,73.0,100.0,997.6,984.5,63.0,100.0,80.0,Heavy thunderstorm with hail,103.28,N/A,LKO-80-97-29062502284729,False,2025-06-29 02:28:47.294467,False,False,transformed,20250629022847300


0    96.0
1    80.0
2     3.0
3     3.0
4     3.0
5    80.0
6    80.0
7     3.0
8     3.0
9    80.0
Name: weather_code, dtype: float64

In [323]:
#DATA VALIDATION


def validate(df):
    errors = []

    # 1. Schema / Column Presence
    required_columns = [
        'city', 'temperature_celcius', 'humidity_%', 'weather_code',
        'unique_id', 'created_at', 'is_deleted', 'processing_status', 'batch_id'
    ]
    missing_cols = [col for col in required_columns if col not in df.columns]
    if missing_cols:
        errors.append(f"Missing columns: {missing_cols}")

    # 2. Data Type Checks (spot check)
    expected_types = {
        'city': str,
        'temperature_celcius': float,
        'humidity_%': float,
        'weather_code': (int, float),
        'unique_id': str,
        'created_at': pd.Timestamp,
    }
    for col, dtype in expected_types.items():
        if col in df.columns:
            if not df[col].map(lambda x: isinstance(x, dtype)).all():
                errors.append(f"Column {col} has wrong type(s)")

    # 3. Uniqueness Check
    if not df['unique_id'].is_unique:
        errors.append("Duplicate unique_id found")


    # 5. Range/Logic Check: temperature and humidity
    if df['temperature_celcius'].between(-90, 60).all() is False:
        errors.append("Temperature out of expected range")

    if df['humidity_%'].between(0, 100).all() is False:
        errors.append("Humidity out of expected range")

    # 6. Flag Consistency Check
    if not df[df['snowfall_mm'] > 0]['is_snowfall'].all():
        errors.append("is_snowfall mismatch")

    if not df[(df['rain_mm'] > 0) | (df['showers_mm'] > 0)]['is_rainfall'].all():
        errors.append("is_rainfall mismatch")

    if not df[df['visibility_m'] < 1000]['is_foggy'].all():
        errors.append("is_foggy mismatch")

    # 7. Date Check: created_at not in future
    if (df['created_at'] > datetime.now()).any():
        errors.append("created_at has future dates")


    # Final result
    if errors:
        print("❌ Validation FAILED with the following issues:")
        for err in errors:
            print("-", err)
        return False
    else:
        print("✅ Validation PASSED. Data is ready for downstream.")
        return True
    
validate(df)


✅ Validation PASSED. Data is ready for downstream.


True

In [324]:
#LOAD TO POSTGRES DATABASE AND AUDIT

from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL
from datetime import datetime

if validate(df)==True:
    def load_dataframe_to_postgres(df):
        # --- DB CONFIG ---
        db_user = 'postgres'
        db_password = 'admin'
        db_host = 'localhost'
        db_port = '5432'
        db_name = 'WEATHER AND SPACE ALERT DB'
        schema_name = 'weather_alert'
        table_name = 'weather_hourly_info_initial'
        audit_table = 'weather_alert_audit_trail'

        try:
            # --- Extract batch_id from DataFrame ---
            batch_ids = df['batch_id'].unique()
            if len(batch_ids) != 1:
                raise ValueError("DataFrame must contain exactly one unique batch_id for this operation.")
            batch_id = batch_ids[0]

            # --- Calculate record count ---
            record_count = len(df)

            # --- Build the connection URL ---
            connection_url = URL.create(
                drivername="postgresql+psycopg2",
                username=db_user,
                password=db_password,
                host=db_host,
                port=db_port,
                database=db_name
            )

            # --- Create engine ---
            engine = create_engine(connection_url)

            # --- Ensure schema exists ---
            with engine.begin() as conn:
                conn.execute(text(f'CREATE SCHEMA IF NOT EXISTS "{schema_name}"'))

            # --- Load DataFrame into the database ---
            df.to_sql(
                name=table_name,
                con=engine,
                schema=schema_name,
                if_exists='append', 
                index=False,
                method='multi'
            )

            # --- Update processing_status only for current batch_id ---
            with engine.begin() as conn:
                conn.execute(text(f'''
                    UPDATE "{schema_name}"."{table_name}"
                    SET processing_status = 'loaded to db'
                    WHERE batch_id = :batch_id
                '''), {'batch_id': batch_id})

            # --- Create audit table if not exists ---
            with engine.begin() as conn:
                conn.execute(text(f'''
                    CREATE TABLE IF NOT EXISTS "{schema_name}"."{audit_table}" (
                        batch_id VARCHAR PRIMARY KEY,
                        date DATE NOT NULL,
                        time TIME NOT NULL,
                        record_count INTEGER NOT NULL
                    )
                '''))

            # --- Insert audit trail record ---
            now = datetime.now()
            current_date = now.date()
            current_time = now.time()

            with engine.begin() as conn:
                conn.execute(text(f'''
                    INSERT INTO "{schema_name}"."{audit_table}" (batch_id, date, time, record_count)
                    VALUES (:batch_id, :date, :time, :record_count)
                    ON CONFLICT (batch_id) DO NOTHING
                '''), {
                    'batch_id': batch_id,
                    'date': current_date,
                    'time': current_time,
                    'record_count': record_count
                })

            print(f"✅ Data loaded and 'processing_status' updated in {schema_name}.{table_name} for batch_id={batch_id}")
            print(f"✅ Audit trail updated in {schema_name}.{audit_table} for batch_id={batch_id} with record count {record_count}")

            return True

        except Exception as e:
            print(f"❌ Failed to load data: {e}")
            return False
    
else:
    print('Data load cancelled as validation is not passed')




✅ Validation PASSED. Data is ready for downstream.


In [325]:
load_dataframe_to_postgres(df)

✅ Data loaded and 'processing_status' updated in weather_alert.weather_hourly_info_initial for batch_id=20250629022847300
✅ Audit trail updated in weather_alert.weather_alert_audit_trail for batch_id=20250629022847300 with record count 10


True

In [326]:

if load_dataframe_to_postgres(df)==True:
    print("Execution completed successfully with exit code 0.")

else:
    print(f"Execution failed with exit code 1.")


✅ Data loaded and 'processing_status' updated in weather_alert.weather_hourly_info_initial for batch_id=20250629022847300
✅ Audit trail updated in weather_alert.weather_alert_audit_trail for batch_id=20250629022847300 with record count 10
Execution completed successfully with exit code 0.


In [333]:

df.loc[df['unique_id'] == 'HYD-03-26-29062502284729', 'rain_mm'] = 99


In [334]:
# MONITOR DATA

def detect_extreme_weather_conditions(df):
    extreme_events = []

    for idx, row in df.iterrows():
        events = []

        # 1. Heavy Thunderstorm / Storm
        if (
            row['wind_speed_kmph'] > 50 and
            row['precipitation_%'] > 80 and
            row['precipitation_occured_mm'] > 10 and
            row['cloud_cover_%'] > 90 and
            row['visibility_m'] < 1000
        ):
            events.append("Heavy Thunderstorm / Storm")

        # 2. Cyclone / Severe Windstorm
        if (
            row['wind_speed_kmph'] > 90 and
            row['mean_sea_level_pressure_hpa'] < 990 and
            row['visibility_m'] < 800
        ):
            events.append("Cyclone / Severe Windstorm")

        # 3. Heatwave
        if row['temperature_celcius'] > 45 and row['humidity_%'] < 30:
            events.append("Heatwave")

        # 4. Cold Wave
        if row['temperature_celcius'] < 5 and row['humidity_%'] > 70:
            events.append("Cold Wave")

        # 5. Heavy Rainfall
        if (row['rain_mm'] + row['showers_mm']) > 50:
            events.append("Heavy Rainfall")

        # 6. Snowstorm / Blizzard
        if (
            (row['snowfall_mm'] > 20 or row['snow_depth_mm'] > 50) and
            row['wind_speed_kmph'] > 30
        ):
            events.append("Snowstorm / Blizzard")

        # 7. Dense Fog
        if row['visibility_m'] < 50:
            events.append("Dense Fog")

        if events:
            extreme_events.append({
                "city": row['city'],
                "events": events
            })

    return extreme_events


In [335]:
detect_extreme_weather_conditions(df)

[{'city': 'Delhi', 'events': ['Heavy Rainfall']},
 {'city': 'Hyderabad', 'events': ['Heavy Rainfall', 'Dense Fog']},
 {'city': 'Chennai', 'events': ['Dense Fog']}]

In [339]:
# GENERATE ALERT

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_extreme_weather_alert_email(alerts, recipient_email, sender_email, sender_password):
    if not alerts:
        print("No extreme alerts to send.")
        return

    # Build the alert message
    subject = "⚠️ Extreme Weather Alert Report"
    body = "Dear User,\n\nThis is an important weather alert from our monitoring system. The following major cities are currently experiencing extreme weather conditions:\n\n"
    
    for alert in alerts:
        city = alert['city']
        events = ", ".join(alert['events'])
        body += f"• {city}: {events}\n"
    
    body += "\n\nPlease take necessary precautions.\nThis is an automated notification. Please do not reply to this message. \n\nRegards,\nWeather Alert System"

    # Create email
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = ", ".join(recipient_email)
    message['Subject'] = subject
    message.attach(MIMEText(body, 'plain'))
    message['X-Priority'] = '1'
    message['X-MSMail-Priority'] = 'High'
    message['Importance'] = 'High'


    # Send the email
    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, recipient_email, message.as_string())
        print(f"✅ Alert email sent to {recipient_email}")
    except Exception as e:
        print(f"❌ Failed to send email: {e}")

#call function
alerts=detect_extreme_weather_conditions(df)
send_extreme_weather_alert_email(
    alerts,
    recipient_email=["argha56562@gmail.com"],
    sender_email="dev.mailer.shandro@gmail.com",
    sender_password="ujoa wpga mggc azlh"
)


✅ Alert email sent to ['argha56562@gmail.com']


In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_extreme_weather_alert_email(alerts, recipient_email, sender_email, sender_password):
    if not alerts:
        print("No extreme alerts to send.")
        return

    subject = "IMPORTANT ALERT: Severe Weather Conditions Reported"


    # Build HTML body with minimal styling
    html_body = """
    <html>
      <body style="font-family: Arial, sans-serif; color: #333;">
        <div style="border-bottom: 4px solid red; padding-bottom: 10px; margin-bottom: 20px;">
          <h1 style="color: red; font-size: 20px; margin: 0;">⚠️ Extreme Weather Alert Report</h1>
        </div>
        <p>Dear User,</p>
        <p>This is an important weather alert from our monitoring system. The following major cities are currently experiencing extreme weather conditions:</p>
        
        <table style="border-collapse: collapse; width: 100%; max-width: 600px;">
          <thead>
            <tr style="background-color: #f2f2f2;">
              <th style="border: 1px solid #ddd; padding: 8px; text-align: left;">City</th>
              <th style="border: 1px solid #ddd; padding: 8px; text-align: left;">Conditions</th>
            </tr>
          </thead>
          <tbody>
    """

    for alert in alerts:
        city = alert['city']
        events = ", ".join(alert['events'])
        html_body += f"""
            <tr>
              <td style="border: 1px solid #ddd; padding: 8px;">{city}</td>
              <td style="border: 1px solid #ddd; padding: 8px;">{events}</td>
            </tr>
        """

    html_body += """
          </tbody>
        </table>

        <p>Please take necessary precautions.</p>
        <p><em>This is an automated notification. Please do not reply to this message.</em></p><br>
        <p>Regards,<br>Weather Alert System</p>
      </body>
    </html>
    """

    message = MIMEMultipart("alternative")
    message['From'] = sender_email
    message['To'] = ", ".join(recipient_email)
    message['Subject'] = subject
    message['X-Priority'] = '1'
    message['X-MSMail-Priority'] = 'High'
    message['Importance'] = 'High'

    # Attach plain text fallback (optional but good practice)
    plain_body = "Dear User,\n\nThis is an important weather alert from our monitoring system.\n\n"
    for alert in alerts:
        city = alert['city']
        events = ", ".join(alert['events'])
        plain_body += f"{city}: {events}\n"
    plain_body += "\nPlease take necessary precautions.\nThis is an automated notification. Please do not reply.\n\nRegards,\nWeather Alert System"
    
    message.attach(MIMEText(plain_body, 'plain'))
    message.attach(MIMEText(html_body, 'html'))

    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, recipient_email, message.as_string())
        print(f"✅ Alert email sent to {recipient_email}")
    except Exception as e:
        print(f"❌ Failed to send email: {e}")



#call function
alerts=detect_extreme_weather_conditions(df)
send_extreme_weather_alert_email(
    alerts,
    recipient_email=["arghamitra4626@gmail.com"],
    sender_email="dev.mailer.shandro@gmail.com",
    sender_password="ujoa wpga mggc azlh"
)



✅ Alert email sent to ['pragya152002@gmail.com']
